In [8]:
import os

In [39]:
pip install ddgs


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.1/863.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ddgs]
Note: you may need to restart the kernel to use updated packages.


In [11]:

# from duckduckgo_search import DDGS


from ddgs import DDGS

In [12]:
from langchain.chat_models import init_chat_model


In [22]:
pip install langchain-openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_openai import ChatOpenAI

In [14]:
from deepagents import create_deep_agent

In [20]:
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"


In [21]:
# ---------- Search Tool ----------
def internet_search(query: str, max_results: int = 5):
    """Run a web search using DuckDuckGo (no paid API needed).
    Returns search results with titles and URLs for better context."""
    print(f"🔍 Searching for: {query}")  # Debug: see what queries are being made
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        # Return more context including title and URL
        formatted_results = []
        for r in results:
            formatted_results.append(
                f"Title: {r.get('title', 'N/A')}\n"
                f"URL: {r.get('href', 'N/A')}\n"
                f"Content: {r.get('body', 'N/A')}\n"
                f"---"
            )
        return "\n".join(formatted_results)


In [ ]:
# # ---------- Search Tool ----------
# def internet_search(query: str, max_results: int = 8, use_news: bool = True):
#     """Run a web search using DuckDuckGo (no paid API needed).
    
#     Args:
#         query: Search query string
#         max_results: Number of results to return (default 8)
#         use_news: If True, searches news sources for recent info (default True)
    
#     Returns search results with titles, URLs and publication dates for better context."""
#     print(f"🔍 Searching for: {query} {'[NEWS]' if use_news else '[WEB]'}")
    
#     with DDGS() as ddgs:
#         # Try news search first for recent events, fall back to text search
#         try:
#             if use_news:
#                 results = list(ddgs.news(query, max_results=max_results))
#             else:
#                 results = list(ddgs.text(query, max_results=max_results))
#         except Exception as e:
#             print(f"⚠️ News search failed, falling back to text search: {e}")
#             results = list(ddgs.text(query, max_results=max_results))
        
#         if not results:
#             return "No results found."
        
#         # Return more context including title, URL, and date
#         formatted_results = []
#         for i, r in enumerate(results, 1):
#             date = r.get('date', 'Date unknown')
#             formatted_results.append(
#                 f"Result {i}:\n"
#                 f"Title: {r.get('title', 'N/A')}\n"
#                 f"Date: {date}\n"
#                 f"URL: {r.get('href', r.get('url', 'N/A'))}\n"
#                 f"Content: {r.get('body', r.get('excerpt', 'N/A'))}\n"
#                 f"---"
#             )
#         return "\n".join(formatted_results)


In [ ]:
# # ---------- Search Tool ----------
# def internet_search(query: str, max_results: int = 8, use_news: bool = True):
#     """Run a web search using DuckDuckGo (no paid API needed).
    
#     Args:
#         query: Search query string
#         max_results: Number of results to return (default 8)
#         use_news: If True, searches news sources for recent info (default True)
    
#     Returns search results with titles, URLs and publication dates for better context."""
#     print(f"🔍 Searching for: {query} {'[NEWS]' if use_news else '[WEB]'}")
    
#     with DDGS() as ddgs:
#         # Try news search first for recent events, fall back to text search
#         try:
#             if use_news:
#                 results = list(ddgs.news(query, max_results=max_results))
#             else:
#                 results = list(ddgs.text(query, max_results=max_results))
#         except Exception as e:
#             print(f"⚠️ News search failed, falling back to text search: {e}")
#             results = list(ddgs.text(query, max_results=max_results))
        
#         if not results:
#             return "No results found."
        
#         # Return more context including title, URL, and date
#         formatted_results = []
#         for i, r in enumerate(results, 1):
#             date = r.get('date', 'Date unknown')
#             formatted_results.append(
#                 f"Result {i}:\n"
#                 f"Title: {r.get('title', 'N/A')}\n"
#                 f"Date: {date}\n"
#                 f"URL: {r.get('href', r.get('url', 'N/A'))}\n"
#                 f"Content: {r.get('body', r.get('excerpt', 'N/A'))}\n"
#                 f"---"
#             )
#         return "\n".join(formatted_results)


In [22]:
# ---------- System Instructions ----------
research_instructions = """You are an expert AI researcher with access to real-time web search.
Your job is to perform thorough research and provide clear, factual summaries based on the MOST RECENT information available.

⚠️ CRITICAL: Your training data is SEVERELY OUTDATED. You do NOT know:
- The current date or year
- Any events from 2024 or later
- Current champions, winners, or titleholders in ANY sport or field

MANDATORY RULES: 
- ALWAYS use 'internet_search' tool for ANY time-sensitive information
- NEVER trust your training data for "current" or "recent" questions
- When searching, look at the "Date:" field in results - this shows when articles were published
- Always prioritize the most recent dates in search results
- If asked about "current" year/date, search for "what is today's date" or similar
- Always cite the publication date and source in your answer
- If search results seem outdated, try searching with "2025" or "November 2025" in your query
"""


In [23]:
# Create the LLM with OpenRouter configuration
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_BASE_URL"]
)

# Create agent - check if deepagents accepts model parameter
agent = create_deep_agent(
    tools=[internet_search],
    system_prompt=research_instructions,
    model=llm  # Try 'model' instead of 'llm'
)

/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object)

In [24]:
result = agent.invoke({"messages": [{"role": "user", "content": " Who is UFC welterweight champion?"}]})
print(result["messages"][-1].content)

🔍 Searching for: current UFC welterweight champion
🔍 Searching for: who is UFC welterweight champion 2025
As of November 15, 2025, the current UFC Welterweight Champion is **Islam Makhachev**, who won the title by defeating Jack Della Maddalena at UFC 322. Prior to this, Della Maddalena had captured the title by defeating Belal Muhammad at UFC 315 on May 10, 2025, showcasing a dominating performance. 

For reference, this information comes from a report published on November 17, 2025, on ITN WWE ([source](https://www.itnwwe.com/mma/current-ufc-champions-titleholders/)).


In [63]:
result = agent.invoke({"messages": [{"role": "user", "content": "First check what year this is. then, Check the most recent news and tell me, Who won the UFC welterweight title recently?"}]})
print(result["messages"][-1].content)

🔍 Searching for: UFC welterweight title winner latest news
🔍 Searching for: current year


/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_config.py:150: ResourceWarning: unclosed <ssl.SSLSocket fd=87, family=2, type=1, proto=0, laddr=('192.168.1.95', 55312), raddr=('20.204.244.192', 443)>
  return self.config_dict[name]


The current year is **2025**.

Recently, **Islam Makhachev** won the UFC welterweight title at UFC 322 on **November 15, 2025**. He secured the title by dominating his opponent, **Jack Della Maddalena**, during the main event held at Madison Square Garden in New York City. Makhachev's victory solidified his status as the UFC's pound-for-pound king, marking him as a champion across two weight classes. 

For more details, you can check the articles on ESPN and MMA Junkie:
- [UFC 322 results and analysis - ESPN](https://www.espn.com/mma/story/_/id/46949041/ufc-322-live-results-analysis-jack-della-maddalena-islam-makhachev-valentina-shevchenko-zhang-weili)
- [UFC 322 results: Islam Makhachev dominates - MMA Junkie](https://mmajunkie.usatoday.com/story/sports/ufc/2025/11/16/ufc-322-islam-makhachev-vs-jack-della-maddalena-results-new-champion/87302290007/)


In [3]:
# ============================================
# RAG SYSTEM - Chat with Your Research Papers
# ============================================
# Install required packages for RAG
!pip install pypdf chromadb langchain-community langchain-text-splitters


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 959.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.1 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [langchain-community]ngchain-community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
spyder 5.4.3 requires pyqt5<5.16, which is not installed.
spyder 5.4.3 requires pyqtwebengine<5.16, which is not installed.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow 2.18.0 requires 

In [25]:
# ---------- RAG Imports ----------
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from pathlib import Path
import glob


In [9]:
import os

In [26]:
# ---------- Initialize Embeddings ----------
# Using OpenAI embeddings (same API key as your LLM)
embeddings = OpenAIEmbeddings(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_BASE_URL"]
)


In [ ]:
# ---------- Load and Process PDF Documents ----------
def load_pdfs_from_directory(directory_path: str = "./pdfs"):
    """
    Load all PDF files from a directory and process them.
    
    Args:
        directory_path: Path to folder containing PDF files (default: ./pdfs)
    
    Returns:
        List of document chunks ready for vector storage
    """
    # Create directory if it doesn't exist
    Path(directory_path).mkdir(exist_ok=True)
    
    # Find all PDF files
    pdf_files = glob.glob(f"{directory_path}/*.pdf")
    
    if not pdf_files:
        print(f" No PDF files found in {directory_path}")
        print(f" Please add your research paper PDFs to the '{directory_path}' folder")
        return []
    
    print(f" Found {len(pdf_files)} PDF file(s):")
    for pdf in pdf_files:
        print(f"   - {Path(pdf).name}")
    
    # Load all PDFs
    all_documents = []
    for pdf_path in pdf_files:
        print(f"\n Loading: {Path(pdf_path).name}...")
        try:
            loader = PyPDFLoader(pdf_path)
            documents = loader.load()
            # Add metadata about source file
            for doc in documents:
                doc.metadata['source_file'] = Path(pdf_path).name
            all_documents.extend(documents)
            print(f"   ✓ Loaded {len(documents)} pages")
        except Exception as e:
            print(f"   ✗ Error loading {pdf_path}: {e}")
    
    # Split documents into chunks
    print(f"\n Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,      # Size of each chunk
        chunk_overlap=200,    # Overlap between chunks for context
        length_function=len,
    )
    
    chunks = text_splitter.split_documents(all_documents)
    print(f"   ✓ Created {len(chunks)} chunks")
    
    return chunks


In [ ]:
# ---------- Create Vector Store ----------
def create_vector_store(chunks, persist_directory: str = "./chroma_db"):
    """
    Create a ChromaDB vector store from document chunks.
    
    Args:
        chunks: List of document chunks
        persist_directory: Where to store the vector database
    
    Returns:
        Chroma vector store object
    """
    if not chunks:
        print(" No documents to process!")
        return None
    
    print(f"\n Creating embeddings and vector store...")
    # print(f"Time depends on the document size...")
    
    # Create vector store
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    
    print(f" Vector store created and saved to '{persist_directory}'")
    print(f" Total documents indexed: {len(chunks)}")
    
    return vectorstore




In [29]:
# Load your PDFs and create vector store
# Change "./pdfs" to the path where your research papers are stored
chunks = load_pdfs_from_directory("./pdfs")

📚 Found 2 PDF file(s):
   - IEEE__Do_not_Change__working_bias_preparing__finalzied.pdf
   - Integrating_Large_Language_Models_and_Knowledge_Graphs__Ethical__Social__and_Technical_Challenges.pdf

📄 Loading: IEEE__Do_not_Change__working_bias_preparing__finalzied.pdf...
   ✓ Loaded 21 pages

📄 Loading: Integrating_Large_Language_Models_and_Knowledge_Graphs__Ethical__Social__and_Technical_Challenges.pdf...
   ✓ Loaded 23 pages

✂️ Splitting documents into chunks...
   ✓ Created 210 chunks


In [30]:
# Load your PDFs and create vector store
# Change "./pdfs" to the path where your research papers are stored
# chunks = load_pdfs_from_directory("./pdfs")
vectorstore = create_vector_store(chunks) if chunks else None


🔢 Creating embeddings and vector store...
   This may take a moment depending on document size...
   ✓ Vector store created and saved to './chroma_db'
   ✓ Total documents indexed: 210


In [ ]:
# ---------- Document Retrieval Tool ----------
def search_documents(query: str, k: int = 5):
    """
    Search your research papers for relevant information.
    
    Args:
        query: Your search question
        k: Number of relevant chunks to retrieve (default: 5)
    
    Returns:
        Formatted string with relevant document excerpts and sources
    """
    if vectorstore is None:
        return " No documents loaded. Please load PDFs first using load_pdfs_from_directory()"
    
    print(f" Searching documents for: '{query}'")
    
    # Search for similar chunks
    results = vectorstore.similarity_search_with_score(query, k=k)
        #usually distance, not similarity
        #Lower score = more similar (because it’s distance)
    if not results:
        return "No relevant information found in the documents."
    
    # Format results
    formatted_results = []
    for i, (doc, score) in enumerate(results, 1):
        source = doc.metadata.get('source_file', 'Unknown')
        page = doc.metadata.get('page', 'N/A')
        
        formatted_results.append(
            f"--- Result {i} (Relevance: {score:.3f}) ---\n"
            f"Source: {source} (Page {page})\n"
            f"Content:\n{doc.page_content}\n"
        )
    
    return "\n".join(formatted_results)


In [32]:
# ---------- RAG Agent System Instructions ----------
rag_instructions = """You are an expert research assistant with access to a knowledge base of research papers.

Your capabilities:
1. You can search through uploaded research papers using the 'search_documents' tool
2. You can search the web using 'internet_search' for additional context
3. You provide accurate, well-sourced answers based on the documents

IMPORTANT RULES:
- ALWAYS use 'search_documents' first when answering questions about the research papers
- Cite specific sources (paper name and page number) in your answers
- If information isn't in the documents, you can use 'internet_search' for additional context
- Be precise and quote relevant sections when appropriate
- If asked about something not in the documents, clearly state that
- Combine information from multiple sources when relevant
"""


In [ ]:
# ---------- Create RAG Agent ----------
# Create LLM
rag_llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_BASE_URL"]
)

# Create agent with both document search and web search tools
rag_agent = create_deep_agent(
    tools=[search_documents, internet_search],
    system_prompt=rag_instructions,
    model=rag_llm
)

print("RAG Agent created successfully!")
print("\n You can now ask questions about your research papers!")
print("   Example: 'What are the main findings in the papers?'")
print("   Example: 'Summarize the methodology used in the research'")


✅ RAG Agent created successfully!

📝 You can now ask questions about your research papers!
   Example: 'What are the main findings in the papers?'
   Example: 'Summarize the methodology used in the research'


/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/abhashshrestha/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object)

In [ ]:
# ---------- Test the RAG System ----------
# Example query - change this to ask about your research papers!
query = """For each research paper, please provide:
1. The paper title
2. The main topics and findings from that specific paper

Organize your response by separating the findings for each paper with clear headings using the paper titles. Make sure to clearly distinguish between the two papers."""

if vectorstore is not None:
    result = rag_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    print("\n" + "="*60)
    print("AGENT RESPONSE:")
    print("="*60)
    print(result["messages"][-1].content)
else:
    print("⚠️ Please load your PDFs first! Add them to the './pdfs' folder and run the load_pdfs_from_directory() cell.")



AGENT RESPONSE:
It seems that there are currently no research papers available for review. If you have specific papers you would like me to analyze or any other queries related to research topics or findings, please let me know!


In [35]:
# ---------- Diagnostic: Check Current Status ----------
print("🔍 Checking RAG System Status...")
print("\n1. Checking for PDFs folder:")
import os
if os.path.exists("./pdfs"):
    pdf_files = [f for f in os.listdir("./pdfs") if f.endswith('.pdf')]
    print(f"   ✓ Found {len(pdf_files)} PDF file(s):")
    for pdf in pdf_files:
        print(f"      - {pdf}")
else:
    print("   ✗ PDFs folder not found!")

print("\n2. Checking for vector store:")
if os.path.exists("./chroma_db"):
    print("   ✓ Vector store directory exists!")
    try:
        # Try to load it
        test_vectorstore = Chroma(
            persist_directory="./chroma_db",
            embedding_function=embeddings
        )
        count = test_vectorstore._collection.count()
        print(f"   ✓ Vector store loaded! Contains {count} document chunks")
        print("   💡 Run the next cell to load it properly!")
    except Exception as e:
        print(f"   ⚠️ Vector store exists but couldn't load: {e}")
        print("   💡 You may need to recreate it by running Cell 19")
else:
    print("   ✗ Vector store not found!")
    print("   💡 Run Cell 19 to create it!")

print("\n3. Checking vectorstore variable:")
if 'vectorstore' in globals() and vectorstore is not None:
    print("   ✓ vectorstore variable is set!")
else:
    print("   ✗ vectorstore variable is None or not set")
    print("   💡 This is why the agent can't find documents!")


🔍 Checking RAG System Status...

1. Checking for PDFs folder:
   ✓ Found 2 PDF file(s):
      - IEEE__Do_not_Change__working_bias_preparing__finalzied.pdf
      - Integrating_Large_Language_Models_and_Knowledge_Graphs__Ethical__Social__and_Technical_Challenges.pdf

2. Checking for vector store:
   ✓ Vector store directory exists!
   ✓ Vector store loaded! Contains 210 document chunks
   💡 Run the next cell to load it properly!

3. Checking vectorstore variable:
   ✓ vectorstore variable is set!


/var/folders/9q/w7dg5r0s76sb6z8r8j3k4ll80000gn/T/ipykernel_59955/2508103239.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  test_vectorstore = Chroma(


In [36]:
# ---------- FIX: Load Existing Vector Store ----------
# If you already created a vector store, run this cell to load it

try:
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )
    
    # Verify it has documents
    count = vectorstore._collection.count()
    print(f"✅ Vector store loaded successfully!")
    print(f"📊 Contains {count} document chunks")
    print(f"\n🎉 Your RAG system is now ready to use!")
    print(f"   You can now ask questions about your research papers.")
    
except Exception as e:
    print(f"❌ Error loading vector store: {e}")
    print(f"\n💡 Try running Cell 19 again to recreate the vector store from your PDFs")


✅ Vector store loaded successfully!
📊 Contains 210 document chunks

🎉 Your RAG system is now ready to use!
   You can now ask questions about your research papers.


In [38]:
# ---------- Test Again After Loading Vector Store ----------
# Now that vectorstore is loaded, try asking a question!

query = """For each research paper, please provide:
1. The paper title
2. The main topics and findings from that specific paper

Organize your response by separating the findings for each paper with clear headings using the paper titles. Make sure to clearly distinguish between the two papers."""

if 'vectorstore' in globals() and vectorstore is not None:
    print(f"📝 Asking: {query}\n")
    result = rag_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    print("\n" + "="*60)
    print("AGENT RESPONSE:")
    print("="*60)
    print(result["messages"][-1].content)
else:
    print("⚠️ Vector store not loaded. Please run Cell 28 first!")


📝 Asking: For each research paper, please provide:
1. The paper title
2. The main topics and findings from that specific paper

Organize your response by separating the findings for each paper with clear headings using the paper titles. Make sure to clearly distinguish between the two papers.

📖 Searching documents for: 'research'
📖 Searching documents for: 'language models and knowledge graphs'
📖 Searching documents for: 'AI bias'

AGENT RESPONSE:
Here are the findings from the two research papers I found relevant based on the topics of AI bias and language models:

### 1. Towards FAIR AI: A Survey of Trends and Knowledge Graph-Enhanced Bias Mitigation
- **Main Topics**:
  - This paper provides a comprehensive survey of trends in AI bias, particularly focusing on disparities between developed and developing countries in AI research.
  - It examines the landscape of AI bias research, exploring how biases manifest across different stages of AI development, from data collection to model 

In [ ]:
# ---------- Load Existing Vector Store (Optional) ----------
# If you've already created a vector store, you can load it without re-processing PDFs
# Uncomment and run this cell instead of the create_vector_store cell above

# vectorstore = Chroma(
#     persist_directory="./chroma_db",
#     embedding_function=embeddings
# )
# print("✅ Loaded existing vector store!")


# 📚 RAG System Setup Instructions

## Step 1: Add Your PDFs
1. Create a folder called `pdfs` in the same directory as this notebook
2. Copy your research paper PDFs into that folder

## Step 2: Run the Setup Cells
1. Run Cell 15: Install packages
2. Run Cell 16: Import libraries  
3. Run Cell 17: Initialize embeddings
4. Run Cell 18: Load PDF function (defined)
5. Run Cell 19: **Load your PDFs and create vector store** ⭐
6. Run Cell 20: Document search tool (defined)
7. Run Cell 21: Agent instructions (defined)
8. Run Cell 22: **Create the RAG agent** ⭐

## Step 3: Ask Questions!
Use Cell 23 or create your own queries to ask about your research papers.

## Tips:
- The vector store is saved to `./chroma_db` - you don't need to reload PDFs every time
- Use Cell 24 to reload an existing vector store
- The agent can search both your documents AND the web for comprehensive answers
